In [48]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
import pyautogui
from bs4 import BeautifulSoup
import time
import pandas as pd
import random
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException

## Scraping and Pickling

I really wanted to incorporate some web scraping into this project. After comparing several datasets, I found a dataset from the FAO for commodities that looked great and one from EM-DAT for natural disasters that seemed comprehensive. Unfortunately, to access EM-DAT, you need a password and entering that from a script and then scraping felt sketchy. The FAO dataset is hugely comprehensive and would not load a view on the webpage, so I opted to just download it. 

FAOSTAT - Commodities Data: 
Globally and by country
https://www.fao.org/faostat/en/#rankings/countries_by_commodity

In [41]:
# Globally:
quick_pickle = pd.read_csv('data/FAOSTAT_data_2.csv')
quick_pickle.to_pickle('quick_pickle.pkl')

In [42]:
# By Country:
bread_and_butter = pd.read_csv('data/Production_Crops_Livestock_E_All_Data.csv',encoding='ISO-8859-1')
bread_and_butter.to_pickle('bread_and_butter.pkl')

/var/folders/ly/bxd94jrj5f1ffy3zk8_yklrh0000gn/T/ipykernel_3598/2437603376.py:2: DtypeWarning: Columns (11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59,62,65,68,71,74,77,80,83,86,89,92,95,98,101,104,107,110,113,116,119,122,125,128,131,134,137,140,143,146,149,152,155,158,161,164,167,170,173,176,179,182,185,188,191,194) have mixed types. Specify dtype option on import or set low_memory=False.
  bread_and_butter = pd.read_csv('data/Production_Crops_Livestock_E_All_Data.csv',encoding='ISO-8859-1')


In [43]:
# Trade Indices:
#https://www.fao.org/faostat/en/#data/TI
trade = pd.read_csv('data/Trade_Indices_E_All_Data.csv',encoding='ISO-8859-1')
trade.to_pickle('trade.pkl')


EM-DAT dataset:

In [45]:
nat_dillsaster = pd.read_excel('data/public_emdat_all1.xlsx',engine='openpyxl')
nat_dillsaster.to_pickle('nat_dillsaster.pkl')

I did find a GDP dataset I wanted to incoporate into my analysis. I also ran into an issue on this page. It is a scrolling table and 'Inspect Elements' doesn't show the html for the entire table. You have to slide the scroll bar to load the rest of the html, presumably with selenium. This seemed unnecessarily complicated, so I opted for a workaround that is only maybe considered successful scraping. 

I used selenium to print the whole table. I then tried various ways to unlock my accessibility features to hit 'esc' to close the Print Dialogue Box, because below it is a super simple and complete table of all the GDP data I need. (I know we said scraping comes to a stop if there is a popup, but my curiosity clearly needed to see for itself.) The user must hit cancel on the print dalogue box, and then the code switches to that other webpage(since the print preview is generated by Chrome) and scrapes it.

In [51]:
# chromedriver_path = '/Users/annaleoni/Desktop/Final/pkl_jar/chromedriver-mac-arm64/chromedriver'
# service = Service(chromedriver_path)
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
# GO TO WEBSITE
driver.get('https://wits.worldbank.org/CountryProfile/en/country/by-country/startyear/ltst/endyear/ltst/indicator/NY-GDP-MKTP-CD')

In [35]:
# Navigate to print button and select all:
try:
    print_button_1 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "DataPrint")))
    print_button_1.click()
    time.sleep(2)

    print_all_radio_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, "All")))
    print_all_radio_button.click()
    time.sleep(3)

    print_button_2 = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, "DataPrintBtn")))
    print_button_2.click()

    time.sleep(5)

    # pyautogui.press('esc')
    # alternative:
    actions = ActionChains(driver)
    actions.send_keys(Keys.ESCAPE).perform()
    time.sleep(5)

    driver.switch_to.window(driver.window_handles[-1])
    print("Current URL:", driver.current_url)
    print("Page title:", driver.title)

    # now i'm at the print table layout:
    html_content = driver.page_source
    print(html_content[:100])
    soup = BeautifulSoup(html_content, 'html.parser')

    table = soup.find('table')

    if table is None:
        print("No table found in HTML content")
    else:

        #get headers
        headers = ['Country Name'] + [th.text.strip() for th in table.find_all('th')[1:]]
        print(headers, " <-- Headers")

        data=[]
        for row in table.find_all('tr')[1:]:
            cols = row.find_all('td')
            if cols:
                country_name = cols[0].text.strip()
                gdp_vals = [col.text.strip() for col in cols[1:]]
                data.append([country_name] + gdp_vals)
        print("data", data[:3])

        # dataframe it
        gdp_df = pd.DataFrame(data, columns=headers)
        print("shape: ", gdp_df.shape)
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    driver.quit()




Current URL: https://wits.worldbank.org/CountryProfile/en/country/by-country/startyear/ltst/endyear/ltst/indicator/NY-GDP-MKTP-CD
Page title: Indicators by Trading Partner - Print
<html><head>
<meta charset="utf-8">
<title>Indicators by Trading Partner - Print </title>
<style>
bo
['Country Name', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', 'TooltipID']  <-- Headers
data [['United States', '5,236,438,000,000.00', '5,641,580,000,000.00', '5,963,144,000,000.00', '6,158,129,000,000.00', '6,520,327,000,000.00', '6,858,559,000,000.00', '7,287,236,000,000.00', '7,639,749,000,000.00', '8,073,122,000,000.00', '8,577,554,457,000.00', '9,062,818,202,000.00', '9,631,174,489,000.00', '10,250,947,997,000.00', '10,581,929,774,000.00', '10,929,112,955,000.00', '11,456,442,041,0

In [36]:
gdp_df.head()
gdp_df.to_pickle('GarlicDillPickle.pkl')